<a href="https://colab.research.google.com/github/n-goyal/python-test/blob/master/Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic

## Libraries & tools

In [0]:
#Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import date

In [81]:
#data collection
!pip install nsepy
from nsepy import get_history

### Data Visualization - Part II

## Data collection

In [0]:
tcs_data = get_history(symbol='TCS',
                    start=date(2018,4,1), 
                    end=date(2019,2,28))

infy_data = get_history(symbol='INFY',
                    start=date(2018,4,1), 
                    end=date(2019,2,28))

In [84]:
infy_data.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2018-04-02,INFY,EQ,1131.80,1141.00,1149.55,1121.3,1136.70,1137.15,1135.86,4036351,4.584717e+14,142078,2960225,0.7334
2018-04-03,INFY,EQ,1137.15,1134.70,1143.55,1128.1,1139.40,1140.45,1135.21,2038584,2.314227e+14,114034,1366638,0.6704
2018-04-04,INFY,EQ,1140.45,1144.00,1144.55,1120.0,1122.70,1124.20,1131.81,2406651,2.723879e+14,137029,1472105,0.6117
2018-04-05,INFY,EQ,1124.20,1139.55,1151.30,1129.1,1146.05,1147.55,1140.71,3881772,4.427993e+14,101745,2899162,0.7469
2018-04-06,INFY,EQ,1147.55,1143.00,1146.00,1122.1,1127.55,1127.00,1128.81,2968871,3.351285e+14,137277,1891236,0.6370


In [0]:
#dropping irrelavent columns
def drop(data):
  return data.drop(['Symbol', 'Series', 'Open', 'High', 'Low', 'Last', 'VWAP', 'Turnover', 'Trades', '%Deliverble', 'Deliverable Volume', 'Prev Close'], axis = 1)

# Part - I

##Volume Shock, Price Shock

    # Time to add some new columns
    # volume_shock = Volume[t] - Volume[t-1]
    # price_shock = Close[t] - Close[t-1]
    # Percentage_Volume_shock = volume_shock / Prev Volume * 100
    # Percentage_close = price_shock / Prev Close * 100

In [0]:
tcs = tcs_data
infy = infy_data

In [0]:
#checking on the shock percentage
def volume_shock(data): 
  data['Volume_diff'] = data['Volume'].diff(1)
  return data['Volume'].diff(1) / data['Volume'] * 100
  
def price_shock(data):
  data['Price_diff'] = data['Close'].diff(1)
  return data['Close'].diff(1) / data['Close'] * 100

def dummy(data):
#   drop(data)
  data['Volume_shock'] = [1 if abs(i)>0 else 0 for i in volume_shock(data)]
  data['Price_shock'] = [1 if abs(i)>2 else 0 for i in price_shock(data)]
  data['PriceV/Svolume'] = [1 if (data['Volume_shock'][i]==0 and data['Price_shock'][i]==1) 
                            else 0 for i in range(len(price_shock(data)))]
  return data

In [0]:
tcs_dummy = dummy(tcs)
infy_dummy = dummy(infy)
tcs_dummy.name = 'TCS'
infy_dummy.name = 'INFY'

In [0]:
infy_dummy

## Moving Averages

### Window = [4,16,28,40,52]

In [0]:
#importing data.. again - to be clearer
tcs = tcs_data
infy = infy_data

In [0]:
# Converting index to datetime index
def to_datetime(data):
  data.index = pd.to_datetime(data.index)
  return data

In [0]:
tcs = to_datetime(tcs)
infy = to_datetime(infy)

    As moving average is to be calculated over weeks
    We'll resample our data over weeks and take moving average

In [0]:
def movingAverage(data, column, window):
    resampled_df = pd.DataFrame()
    x = data[column].resample('W').mean()
    resampled_df['Close'] = x
    for win in window:
        rolling_window = x.rolling(window=win)
        resampled_df[str(win) + 'WeeksWindow'] = rolling_window.mean()
    return resampled_df

In [0]:
window = [4,16,28,40,52]
column = 'Close'
tcs_ma= movingAverage(tcs, column, window)
infy_ma = movingAverage(infy, column, window)

In [0]:
tcs_ma.head()

In [0]:
tcs_ma.plot(figsize=(10,8), title="Moving Averages for TCS")
infy_ma.plot(figsize=(10,8), title="Moving Averages for INFY")

### Window = 10, 75, Considering Holidays

In [0]:
tcs = tcs_data
infy = infy_data
stocks = [tcs, infy]

In [0]:
def dates(data):
  data['Date'] = pd.to_datetime(data.index)
  data['#Week'] = data.Date.dt.weekofyear
  return data
  
#dropping irrelavent columns
def drop(data):
  return data.drop(['Symbol', 
                    'Series', 
                    'Open', 
                    'High', 
                    'Low', 
                    'Last', 
                    'VWAP', 
                    'Turnover', 
                    'Trades', 
                    '%Deliverble', 
                    'Deliverable Volume', 
                    'Prev Close'], axis = 1)

def format_data(data):
  df = drop(data)
  data = dates(df)
  return data

In [0]:
tcs = format_data(tcs)
infy = format_data(infy)

In [0]:
tcs.tail()

In [0]:
# task: rolling mean with window = [10, 75]
# rolling window w/o resampling is throwing nans and doesn't considering the holidays
# Resamping with frequency with Dates
def plot_roll_window(stock, window, stock_name):
  stock = stock.asfreq('D', method='ffill')
  stock['Window '+str(window)] = stock['Close'].rolling(window).mean()
  plt.figure()
  stock['Window '+str(window)].plot()
  plt.title('roll avg for {0} with window = {1} Days'.format(stock_name, window))
  plt.legend()
  return stock

In [0]:
window = [10, 75]
stock = [tcs, infy]
stocks_name = ['tcs', 'infy']
for i in range(len(window)):
  for j in range(len(stocks)):
#     print('roll avg for {0} with window = {1} Days'.format(stocks_name[j], window[i]))
    plot_roll_window(stock[j], window[i], stocks_name[i])
#   print('roll avg for infy with window = ' + str(i) + ' Days')
#   plot_roll_window(infy, i)

# Part - II Visualization

In [0]:
# Mentioned Using Bokeh
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import show, output_notebook

In [0]:
def timeSeriesPlot(dummy_data):
  name = dummy_data.name
  dummy_data['Height'] = abs(dummy_data['Price_diff'])
  dummy_data.index = pd.to_datetime(dummy_data.index)
  
  output_notebook()
  hours_12 = 12 * 60 * 60 * 1000
  p = figure(x_axis_type='datetime', width=1000, height = 300)
  p.rect(dummy_data.index[dummy_data['Volume_shock'] == 1],
           dummy_data.Close[dummy_data['Volume_shock'] == 1],
           hours_12,
           dummy_data.Height[dummy_data['Volume_shock'] == 1],
           color = 'red',
           line_color="black")

  p.rect(dummy_data.index[dummy_data['Volume_shock'] == 0],
           dummy_data.Close[dummy_data['Volume_shock'] == 0],
           hours_12,
           dummy_data.Height[dummy_data['Volume_shock'] == 0],
           color = 'blue',
           line_color="black")
  
  return show(p), output_file('graph {}.html'.format(name))


In [0]:
def volumeLessPriceMovement(dummy_data):
  dummy_data['Height'] = abs(dummy_data['Price_diff'])
  dummy_data.index = pd.to_datetime(dummy_data.index)
  name = dummy_data.name
  output_notebook()
  hours_12 = 12 * 60 * 60 * 1000
  p = figure(x_axis_type='datetime', width=1000, height = 300)
  p.rect(dummy_data.index[dummy_data['PriceV/Svolume'] == 1],
           dummy_data.Close[dummy_data['PriceV/Svolume'] == 1],
           hours_12,
           dummy_data.Height[dummy_data['PriceV/Svolume'] == 1],
           color = 'red',
           line_color="black")

  p.rect(dummy_data.index[dummy_data['PriceV/Svolume'] == 0],
           dummy_data.Close[dummy_data['PriceV/Svolume'] == 0],
           hours_12,
           dummy_data.Height[dummy_data['PriceV/Svolume'] == 0],
           color = 'blue',
           line_color="black")
  
  return show(p), output_file('graph2 {}.html'.format(name))

In [196]:
timeSeriesPlot(tcs_dummy)

Loading BokehJS ...

(None, None)

In [197]:
timeSeriesPlot(infy_dummy)

Loading BokehJS ...

(None, None)

In [198]:
# tcs_dummy[tcs_dummy['Volume_shock'] == 0]
infy_dummy[infy_dummy['Volume_shock'] == 0]

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Volume_diff,Volume_shock,Price_diff,Price_shock,PriceV/Svolume,Height
Date,,,,,,,,,,,,,,,,,,,,
2018-04-02,INFY,EQ,1131.8,1141.0,1149.55,1121.3,1136.7,1137.15,1135.86,4036351,4.584717e+14,142078,2960225,0.7334,NaN,0,NaN,0,0,NaN


In [200]:
volumeLessPriceMovement(tcs_dummy)

Loading BokehJS ...

(None, None)

In [202]:
volumeLessPriceMovement(infy_dummy)

Loading BokehJS ...

(None, None)

    Graph indicates that there wasnot any such movement where price shock occured without volume shock in the acquired data

In [171]:
dummy_data = infy_dummy
dummy_data[dummy_data['PriceV/Svolume'] == 1]

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,Volume_diff,Volume_shock,Price_diff,Price_shock,PriceV/Svolume,Height
Date,,,,,,,,,,,,,,,,,,,,


### Autocorrelation Plot